In [1]:
#check the direction

In [2]:
import os
print(os.getcwd())

C:\Program Files\Microsoft VS Code


In [3]:
import os

# Change to your project directory (update the path as needed)
os.chdir(r"G:\Snow-Analyze-in-Iran")

# Verify the change
print(os.getcwd())

G:\Snow-Analyze-in-Iran


In [5]:
import xarray as xr
import pandas as pd
import pandas as gpd
import regionmask
import numpy as np
import geopandas
import xarray as xr
import xarray as xr
import seaborn as sns
import rioxarray
from shapely.geometry import mapping
import netCDF4
import netCDF4 as nc
from PIL import Image
import os
from netCDF4 import Dataset
import geopandas as gpd
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from scipy.stats import linregress
#from xarrayMannKendall import MannKendall
import pymannkendall as mk
import xarray as xr
import numpy as np
from scipy.stats import kendalltau

In [6]:
data_Snow = xr.open_dataset("srs_cuted_1972-2023.nc")

In [7]:
data_Snow

<xarray.Dataset> Size: 301MB
Dimensions:    (time: 624, longitude: 93, latitude: 81)
Coordinates:
  * time       (time) datetime64[ns] 5kB 1972-01-01 1972-02-01 ... 2023-12-01
  * longitude  (longitude) float32 372B 42.0 42.25 42.5 ... 64.5 64.75 65.0
  * latitude   (latitude) float32 324B 42.0 41.75 41.5 41.25 ... 22.5 22.25 22.0
Data variables:
    asn        (time, latitude, longitude) float64 38MB ...
    snowc      (time, latitude, longitude) float64 38MB ...
    rsn        (time, latitude, longitude) float64 38MB ...
    sde        (time, latitude, longitude) float64 38MB ...
    sd         (time, latitude, longitude) float64 38MB ...
    sf         (time, latitude, longitude) float64 38MB ...
    smlt       (time, latitude, longitude) float64 38MB ...
    tsn        (time, latitude, longitude) float64 38MB ...
Attributes:
    CDI:          Climate Data Interface version 2.0.2 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Sat Jul 06 13:32:45 2024: cdo seldate,1972-01-01,2023-12-01...
    CDO:          Climate Data Operators version 2.0.2 (https://mpimet.mpg.de...

In [3]:
#Variables names
# asn:Snow albedo ((0 - 1)), snowc: snow cover (%), 
#rsn:Snow density (kg m**-3), sde:Snow depth (m), sd:Snow depth (m of water equivalent),
#sf:Snowfall (m of water equivalent), smlt: Snowmelt (m of water equivalent), tsn:Temperature of snow layer (K)

#-------------------------------------------------------------------------------------------------------------#

data_Snow = xr.open_dataset("G:\Snow-Analyze-in-Iran\snow_iran-mon-1950-2023.nc")
snow_cover= data_Snow['snowc']
Snow_albedo = data_Snow['asn']
Snow_density = data_Snow['rsn']
Snow_depth = data_Snow['sde']
Snow_depth_water = data_Snow['sd']
Snow_snowfall = data_Snow['sf']
Snow_snowmelt = data_Snow['smlt']
Snow_Temperature = data_Snow['tsn']
snow_cover

<xarray.DataArray 'snowc' (time: 888, latitude: 201, longitude: 231)>
[41230728 values with dtype=float32]
Coordinates:
  * longitude  (longitude) float32 42.0 42.1 42.2 42.3 ... 64.7 64.8 64.9 65.0
  * latitude   (latitude) float32 42.0 41.9 41.8 41.7 ... 22.3 22.2 22.1 22.0
  * time       (time) datetime64[ns] 1950-01-01 1950-02-01 ... 2023-12-01
Attributes:
    units:      %
    long_name:  Snow cover